In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import numpy as np

In [2]:
low_ratings = pd.read_csv('low_ratings.csv')
klow = pd.read_csv('klow.csv', encoding='latin-1')
kl_ratings = pd.merge(klow,low_ratings)

In [3]:
kl_ratings['song'] = kl_ratings['title'] +' - '+ kl_ratings['artist']
print(kl_ratings.shape)
kl_ratings.head()

(951, 7)


,song_id,title,artist,tempo,user_id,rating,song
0,1,Crazy,4Minute,88.0,1,5,Crazy - 4Minute
1,1,Crazy,4Minute,88.0,6,2,Crazy - 4Minute
2,1,Crazy,4Minute,88.0,8,2,Crazy - 4Minute
3,1,Crazy,4Minute,88.0,28,1,Crazy - 4Minute
4,1,Crazy,4Minute,88.0,65,2,Crazy - 4Minute


In [6]:
numlow_rating = kl_ratings.groupby('song')['rating'].count().reset_index()

In [7]:
numlow_rating.head()

,song,rating
0,724148 - Agust D,21
1,BERMUDA TRIANGLE - ZICO,14
2,Black Mamba - aespa,16
3,Blue Side: Outro - j-hope,18
4,Break all the Rules - CRAVITY,19


In [8]:
numlow_rating.rename(columns={'rating':'no of ratings'}, inplace=True)
final_low = kl_ratings.merge(numlow_rating, on='song')
final_low

,song_id,title,artist,tempo,user_id,rating,song,no of ratings
0,1,Crazy,4Minute,88.000,1,5,Crazy - 4Minute,20
1,1,Crazy,4Minute,88.000,6,2,Crazy - 4Minute,20
2,1,Crazy,4Minute,88.000,8,2,Crazy - 4Minute,20
3,1,Crazy,4Minute,88.000,28,1,Crazy - 4Minute,20
4,1,Crazy,4Minute,88.000,65,2,Crazy - 4Minute,20
...,...,...,...,...,...,...,...,...
946,59,BERMUDA TRIANGLE,ZICO,89.013,260,0,BERMUDA TRIANGLE - ZICO,14
947,59,BERMUDA TRIANGLE,ZICO,89.013,266,2,BERMUDA TRIANGLE - ZICO,14
948,59,BERMUDA TRIANGLE,ZICO,89.013,270,5,BERMUDA TRIANGLE - ZICO,14
949,59,BERMUDA TRIANGLE,ZICO,89.013,281,3,BERMUDA TRIANGLE - ZICO,14


In [9]:
final_low = final_low[final_low['no of ratings']>=5] #only songs above 5 users have rated is qualified, to avoid noise
final_low.shape

(951, 8)

In [10]:
final_low.drop_duplicates(['user_id','song'], inplace=True)
final_low.shape

(925, 8)

In [11]:
low_pivot = final_low.pivot_table(columns='user_id', index='song', values='rating')
low_pivot.fillna(0, inplace=True)

In [12]:
print(low_pivot.shape)
low_pivot

(59, 311)


user_id,1,2,3,4,5,6,7,8,9,10,...,322,323,324,325,326,327,328,329,331,332
song,,,,,,,,,,,,,,,,,,,,,
724148 - Agust D,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BERMUDA TRIANGLE - ZICO,0.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Black Mamba - aespa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
Blue Side: Outro - j-hope,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
Break all the Rules - CRAVITY,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cherry Bomb - NCT 127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
Crazy - 4Minute,5.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DRIP - HINAPIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Don't Call Me - SHINee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
from scipy.sparse import csr_matrix
low_sparse = csr_matrix(low_pivot)
type(low_sparse)

scipy.sparse.csr.csr_matrix

In [14]:
from sklearn.neighbors import NearestNeighbors
low_model = NearestNeighbors(algorithm='brute')

In [15]:
low_model.fit(low_sparse)

NearestNeighbors(algorithm='brute')

In [16]:
distances, suggestions = low_model.kneighbors(low_pivot.iloc[4,:].values.reshape(1,-1), n_neighbors=15)

In [17]:
suggestions

array([[ 4, 14, 33, 10,  8, 30, 26, 50, 20, 32, 17, 37, 11, 42, 15]],
      dtype=int64)

In [18]:
for i in range(len(suggestions)):
    print(low_pivot.index[suggestions[i]])

Index(['Break all the Rules - CRAVITY', 'Film out - BTS', 'MMM - TREASURE',
       'ECLIPSE - GOT7', 'Don't Call Me - SHINee', 'LIMITLESS - NCT 127',
       'Intro: What Am I to You - BTS', 'Trouble - EXO', 'Heart Attack - EXO',
       'Lost - BTS', 'GOODBYE ROAD - iKON', 'On Rainy Days - Beast',
       'Epilogue: Young Forever - BTS', 'Rain - BTS',
       'First Love - After School'],
      dtype='object', name='song')


In [19]:
low_pivot.index[50]

'Trouble - EXO'

In [20]:
np.where(low_pivot.index=='Trouble - EXO')[0][0]

50

In [21]:
def recommend_kmusic_low(song):
    song_id=np.where(low_pivot.index==song)[0][0]
    distances, suggestions = low_model.kneighbors(low_pivot.iloc[song_id,:].values.reshape(1,-1), n_neighbors=15)
    
    for i in range(len(suggestions)):
        print(low_pivot.index[suggestions[i]])

In [22]:
recommend_kmusic_low('Trouble - EXO')

Index(['Trouble - EXO', 'Film out - BTS', 'MMM - TREASURE', 'ECLIPSE - GOT7',
       'Don't Call Me - SHINee', 'LIMITLESS - NCT 127',
       'Intro: What Am I to You - BTS', 'Lost - BTS', 'GOODBYE ROAD - iKON',
       'Whalien 52 - BTS', 'On Rainy Days - Beast',
       'First Love - After School', 'LOSER - BIGBANG',
       'Hellevator - Stray Kids', 'Heart Attack - EXO'],
      dtype='object', name='song')


In [25]:
ratelow = kl_ratings.groupby('song_id')['rating'].count().reset_index()
ratelow

,song_id,rating
0,1,20
1,2,23
2,3,21
3,4,16
4,5,11
5,6,11
6,7,13
7,8,17
8,9,17
9,10,15


In [26]:
low_list = pd.merge(klow,ratelow)
low_list.rename(columns={'rating':'no of ratings'}, inplace=True)
low_list 

,song_id,title,artist,tempo,no of ratings
0,1,Crazy,4Minute,88.000,20
1,2,Under Cover,A.C.E,85.981,23
2,3,724148,Agust D,90.000,21
3,4,Black Mamba,aespa,90.069,16
4,5,First Love,After School,89.990,11
5,6,On Rainy Days,Beast,90.022,11
6,7,FXXK IT,BIGBANG,90.026,13
7,8,LOSER,BIGBANG,89.846,17
8,9,ZUTTER (GD & T.O.P),BIGBANG,90.000,17
9,10,WE LIKE 2 PARTY,BIGBANG,89.952,15


In [27]:
low_list['song'] = low_list['title'] +' - '+ low_list['artist']
low_list 

,song_id,title,artist,tempo,no of ratings,song
0,1,Crazy,4Minute,88.000,20,Crazy - 4Minute
1,2,Under Cover,A.C.E,85.981,23,Under Cover - A.C.E
2,3,724148,Agust D,90.000,21,724148 - Agust D
3,4,Black Mamba,aespa,90.069,16,Black Mamba - aespa
4,5,First Love,After School,89.990,11,First Love - After School
5,6,On Rainy Days,Beast,90.022,11,On Rainy Days - Beast
6,7,FXXK IT,BIGBANG,90.026,13,FXXK IT - BIGBANG
7,8,LOSER,BIGBANG,89.846,17,LOSER - BIGBANG
8,9,ZUTTER (GD & T.O.P),BIGBANG,90.000,17,ZUTTER (GD & T.O.P) - BIGBANG
9,10,WE LIKE 2 PARTY,BIGBANG,89.952,15,WE LIKE 2 PARTY - BIGBANG


In [28]:
import pickle

In [30]:
pickle.dump(low_list,open('klow.pkl','wb'))

In [29]:
low_list['song']

0                            Crazy - 4Minute
1                        Under Cover - A.C.E
2                           724148 - Agust D
3                        Black Mamba - aespa
4                  First Love - After School
5                      On Rainy Days - Beast
6                          FXXK IT - BIGBANG
7                            LOSER - BIGBANG
8              ZUTTER (GD & T.O.P) - BIGBANG
9                  WE LIKE 2 PARTY - BIGBANG
10                          STAY - BLACKPINK
11                                Dope - BTS
12                            I Need U - BTS
13                   Intro: Never Mind - BTS
14                           Jamais Vu - BTS
15             Intro: What Am I to You - BTS
16                            Film out - BTS
17    We are Bulletproof: the Eternal  - BTS
18                                Lost - BTS
19                      Intro: Persona - BTS
20                         Serendipity - BTS
21                          Whalien 52 - BTS
22        

In [31]:
low_list.to_dict()

{'song_id': {0: 1,
  1: 2,
  2: 3,
  3: 4,
  4: 5,
  5: 6,
  6: 7,
  7: 8,
  8: 9,
  9: 10,
  10: 11,
  11: 12,
  12: 13,
  13: 14,
  14: 15,
  15: 16,
  16: 17,
  17: 18,
  18: 19,
  19: 20,
  20: 21,
  21: 22,
  22: 23,
  23: 24,
  24: 25,
  25: 26,
  26: 27,
  27: 28,
  28: 29,
  29: 30,
  30: 31,
  31: 32,
  32: 33,
  33: 34,
  34: 35,
  35: 36,
  36: 37,
  37: 38,
  38: 39,
  39: 40,
  40: 41,
  41: 42,
  42: 43,
  43: 44,
  44: 45,
  45: 46,
  46: 47,
  47: 48,
  48: 49,
  49: 50,
  50: 51,
  51: 52,
  52: 53,
  53: 54,
  54: 55,
  55: 56,
  56: 57,
  57: 58,
  58: 59},
 'title': {0: 'Crazy',
  1: 'Under Cover',
  2: '724148',
  3: 'Black Mamba',
  4: 'First Love',
  5: 'On Rainy Days',
  6: 'FXXK IT',
  7: 'LOSER',
  8: 'ZUTTER (GD & T.O.P)',
  9: 'WE LIKE 2 PARTY',
  10: 'STAY',
  11: 'Dope',
  12: 'I Need U',
  13: 'Intro: Never Mind',
  14: 'Jamais Vu',
  15: 'Intro: What Am I to You',
  16: 'Film out',
  17: 'We are Bulletproof: the Eternal ',
  18: 'Lost',
  19: 'Intro: Per

In [32]:
pickle.dump(low_list.to_dict(),open('klow_dict.pkl','wb'))

In [33]:
pickle.dump(low_pivot,open('low_pivot.pkl','wb'))

In [34]:
pickle.dump(low_model,open('low_model.pkl','wb'))

In [35]:
low_list['tempo']

0     88.000
1     85.981
2     90.000
3     90.069
4     89.990
5     90.022
6     90.026
7     89.846
8     90.000
9     89.952
10    87.959
11    77.058
12    78.000
13    80.000
14    81.000
15    82.000
16    82.000
17    84.000
18    85.000
19    87.000
20    87.000
21    90.000
22    90.000
23    89.000
24    88.000
25    77.501
26    89.974
27    78.028
28    81.590
29    79.454
30    90.000
31    89.924
32    85.000
33    81.000
34    90.000
35    76.022
36    80.000
37    79.889
38    79.000
39    90.000
40    80.000
41    79.000
42    89.000
43    89.994
44    80.000
45    88.999
46    90.000
47    81.509
48    75.025
49    78.027
50    78.216
51    86.000
52    82.026
53    78.169
54    80.007
55    77.565
56    75.010
57    84.000
58    89.013
Name: tempo, dtype: float64